In [2]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import calendar
import datetime as dt

import numpy as np
import fnmatch
import os
from tqdm import tqdm

# Читаем и преобразовываем JSON

In [ ]:
print('Обработка файлов JSON')

firstdata=True
rootDir='json_data'
files=os.listdir(rootDir)
pbar = tqdm(total=len(files))
datas=[]
for file in files:
    pbar.update(1)
    if fnmatch.fnmatch(file, '*.json'):
        with open(rootDir+'/'+file, encoding='utf8') as data_file:
            json_data = json.load(data_file)
            if firstdata:
                _columns=[val['Name']['#value'] for val in json_data['#value']['column']]#Вычленяем колонки
                firstdata=True
            datas.append(pd.DataFrame(data=json_data['#value']['row'], columns=_columns).applymap(lambda cell: cell['#value']))#читаем JSON и вычленяем значения
pbar.close()

print('Преобразуем в датафрейм...')

data=pd.concat(datas, axis=0, join='outer', ignore_index=True)

del json_data       
del datas

print('Готово')

# Преобразование характеристик

In [ ]:
data['Дата']=pd.to_datetime(data['Дата'],format='%Y-%m-%d')#преобразуем дату

#data['ТоварСреднийВес']=data['ТоварСреднийВес'].map(lambda cell: cell if np.isreal(cell) else 0).astype(float)#заполняем пустоты

# Формируем новые характеристики

#параметры даты
#data['Год']=data['Дата'].dt.year
#data['Месяц']=data['Дата'].dt.month
#data['Неделя']=data['Дата'].dt.week
#data['День']=data['Дата'].dt.day
#data['ДеньНедели']=data['Дата'].dt.weekday
#Понедельник-четверг 0..3, пятница - 4, суббота-воскресенье - 5..6
#data['РабочиеВыходные']=data['ДеньНедели'].map(lambda cell: 1 if cell<4 else 2 if cell==4 else 3)

data['НеЗаказыватьУПоставщиков']=data['НеЗаказыватьУПоставщиков'].astype('int')

# Праздники

In [5]:
with open('Праздники 20130101 - 20171231.json', encoding='utf8') as data_file:
    json_data = json.load(data_file)
_col=[val['Name']['#value'] for val in json_data['#value']['column']]
celebrate=pd.DataFrame(data=json_data['#value']['row'], columns=_col).applymap(lambda cell: cell['#value'])
celebrate['Праздник']=pd.to_datetime(celebrate['Праздник'],format='%Y-%m-%d')#преобразуем дату

del json_data

celebrate.to_pickle('celebrate.pickle')

#data['Праздник']=0#Обычный день
#data.loc[data['Дата'].isin(celebrate['Праздник']),'Праздник']=1#праздник


del celebrate

# Удаляем ненужное

In [ ]:
#ata.drop(['Товар'], axis=1, inplace=True)
#data.drop(['Контрагент','МаркетинговаяЛинейка'], axis=1, inplace=True)

# 29 февраля: берем среднее за 28 и 29 февраля

condition29=(data['День'] > 28)&(data['Месяц']==2)#выборка 29 февраля каждого года
for year in data.loc[condition29,'Год'].unique():#Какие года високосные
    condition28=(data['День'] == 28)&(data['Месяц']==2)&(data['Год']==year)#выборка 28 февраля каждого года
    row=data[condition28].iloc[0]#первая строка колонки
    for col in['День','Неделя','ДеньНедели','РабочиеВыходные','Праздник']:#приравниваем характеристики даты
        data.loc[condition29,col]=row[col]

#группируемые колонки
selectColumns=list(set(data.columns)-set(['Количество']))

#среднее по 28 февраля
condition28=(data['День'] == 28)&(data['Месяц']==2)
feb28=data[condition28].groupby(selectColumns)['Количество'].mean().reset_index()

#удаляем из оригинально датасета все 28 февраля
data.drop(data[condition28].index, inplace=True)

#добавляем среднее за 28 февраля
data=pd.concat([data,feb28], axis=0, join='outer', ignore_index=True)

del feb28

## Группировка данных с суммированием количества

In [ ]:
#группируемые колонки
selectColumns=list(set(data.columns)-set(['Количество']))

data=data.groupby(selectColumns)['Количество'].sum().reset_index()
#data.drop(['index'], axis=1, inplace=True)

#Сортировка данных cо сбросом индекса, для корректной последующей разбивки
data.sort_values('Дата',inplace=True)

## Финальная обработка категориальных данных

In [ ]:
#получаем колонки категориальных характеристик
categorical_columns = [c for c in data.columns if data[c].dtype.name == 'object']

#заполняем пустые значения в категориальных данных
data[categorical_columns]=data[categorical_columns].applymap(lambda cell: cell.strip() if bool(cell.strip()) else '<Неопределено>')

# Сохраняем в формате Pickle (Python)

In [ ]:
#проверка на Nan, null
count=0
null_series=data.isnull().any()
for idx in null_series.index:
    if null_series[idx]: 
        count+=1
        print (str(count)+'. '+idx) 
        

if count>0:
    print('Существуют данные null')
    raise SystemExit(2)

data.reset_index().drop(['index'], axis=1).to_pickle('MG_Sales.pickle',compression='gzip')
del data

In [ ]:
print('Завершено',str(dt.datetime.now()))